### 从零开始实现softmax分类

In [38]:
import torch
# noinspection PyTypeChecker
from d2l import torch as d2l
from IPython import display

In [39]:
def get_dataloader_workers():
    """使⽤4个进程来读取数据"""
    return 4

In [40]:
# def load_data_fashion_mnist(batch_size, resize=None):
#     """下载数据集并储存到内存中"""
#
#     trans = [transforms.ToTensor()]  # 通过ToTensor实例将图像数据从PIL类型变换成32位浮点数格式，
#     if resize:
#         """
#             resize可以将图像最小边缩放到指定值， 并且其他边同比例缩放
#             example: torch.Size([32, 1, 32, 32])  -->  torch.Size([32, 1, 64, 64])
#                      [1, 32, 32] --> [cross, height, weight]
#         """
#         trans.insert(0, transforms.Resize(resize))
#     trans = transforms.Compose(trans)
#     mnist_train = torchvision.datasets.FashionMNIST(
#     root='.data', train = True, transform=trans, download=True
# )
#     mnist_test = torchvision.datasets.FashionMNIST(
#         root='.data', train = False, transform=trans, download=True
#     )
#     return  (data.DataLoader(mnist_train, batch_size, shuffle=True,                                                  num_workers=get_dataloader_workers()),
#              data.DataLoader(mnist_test, batch_size, shuffle=True,
#                              num_workers=get_dataloader_workers()))

In [41]:
# train_iter, test_iter = load_data_fashion_mnist(32, resize=64)
# for X, y in train_iter:
#     print(X.shape, X.dtype, y.shape, y.dtype)
#     break

In [42]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [43]:
def softmax(X):
    # softmax实现方式：
    # 1. 对每个项求幂（使⽤exp）；
    # 2. 对每⼀⾏求和（小批量中每个样本是⼀⾏），得到每个样本的规范化常数；
    # 3. 将每⼀⾏除以其规范化常数，确保结果的和为1。
    #
    # tips: 因为one-hot编码是在最后一层做判断，所以要将这784个输入结果求和之后做softmax激活
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)  # 1表示按行求和, keepdim表示在列上不改变矩阵形状
    return X_exp / partition


In [44]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

### 分析net的实现方式
<img src="src/3.4.1_softmax.png" width=400 height=200>

============================

如图，训练应该如下：
o1 = x1w11 + x2w12 + x3w13 + x4w14 + b1,
o2 = x1w21 + x2w22 + x3w23 + x4w24 + b2,
o3 = x1w31 + x2w32 + x3w33 + x4w34 + b3.

对于mnist训练集，输入层一共为784(x1, x2, x3,...,x784)，输出层一共为10(o1, o2, ... ,o10)
那么每一个输出层都应该与相应的784个偏置项所对应
所以W应该是[784, 10]的矩阵来和输入做乘法，之后再将结果输入到softmax激活函数中激活

In [45]:
def net(X):
    # torch.matmul是一个线性函数，传入 W 和 b 后会根据广播机制实现 Y = WX + b
    # 这里reshape可以将 X 按照 W 的维度展开，这样每个输入都会有一个 W
    #
    # X.size() = [batch_size, 1, 28, 28]
    # X.reshape((-1, W.shape[0])) = [batch_size, 748]
    # W.size() = [784, 10]
    # torch.matmul(X.reshape((-1, W.shape[0])), W).size() = [256, 10]
    #
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

def cross_entropy(y_hat, y):
    # 交叉熵函数  -->  Cross Entropy.md
    w = - torch.log(y_hat[range(len(y_hat)), y])
    return w

def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

def evaluate_accuracy(net, data_iter):
    """计算模型在指定数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())  # numel()返回该张量总元素的个数
    return metric[0] / metric[1]

In [46]:
def train_epoch_ch3(net, train_iter, loss, updater):
    if isinstance(net, torch.nn.Module):
        # isinstance(A， B)  -->  A是要判断的对象, B是要判断的类型, 返回布尔值
        net.train()  # train model

    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)  # 传入自定义损失函数
        if isinstance(updater, torch.optim.Optimizer):
            # 使用pytorch内置的优化器和算是函数
            updater.zero_grad()
            l.sum().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            # backward()所接受传入的必须是标量, backward()接受传入的有一个参数grad_tensors, 这个参数是指定哪些方向上的向量需要反向传播
            # 若传入torch.ones(len(X)), 则表示各个方向均需要反向传播, 因为 1 作为常数对反向传播无影响, 若传入 0 则表示该方向不传播
            # backward(torch.ones(len(X))) 是等效于 X.sum().backward() 因为求和操作就相当于X点乘一个全是 1 的矩阵
            l.sum().backward()
            print(X.size())
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())

    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1]/ metric[2]

In [47]:
class Animator:  #@save
    """在动画中绘制数据"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

In [48]:
# noinspection PyShadowingNames
def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):
    """训练模型"""
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch+1, train_metrics+ (test_acc, ))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert 1 >= train_acc > 0.7, train_acc
    assert 1 >= test_acc > 0.7, test_acc

In [ ]:
lr = 0.1
def updater(batch_size):
    # 在sgd函数中, with torch.no_grad() 在该模块下计算出来的tensor的required_grad 设置为False, 这样不会自动求导, 节约内存
    return d2l.sgd([W, b], lr, batch_size)  # sgd 是自动随机梯度下降

num_epochs = 100
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)

### softmax简单实现

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l


batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [ ]:
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))  # 用 Flatten 展平层展开输入

def init_weights(m):
    if type(m) == nn.Linear:
        #  输入一个线性连接层, 然后用_normal方法，以正态分布初始化线性连接层的w
        #  m.weight  --> W  |  m.bias  --> b
        #  nn.init.normal_() --> 正态分布初始化  |  nn.init.zeros_()  --> 初始化成0
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)  # apply()类似于一个for循环, 将fn应用到每一个model中, fn的输入即为每个models


In [ ]:
loss = nn.CrossEntropyLoss()

trainer = torch.optim.SGD(net.parameters(), lr=0.1)
num_epochs = 10
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)